In [1]:
import logging
import numpy as np
import astropy.units as u
from astropy.coordinates.angle_utilities import angular_separation
from astropy.utils import lazyproperty
from regions import CircleSkyRegion
import matplotlib.pyplot as plt
from gammapy.maps import HpxNDMap, Map, RegionNDMap, WcsNDMap, MapAxis, WcsGeom
from gammapy.modeling.models import PointSpatialModel, TemplateNPredModel
# import warnings

PSF_CONTAINMENT = 0.999
CUTOUT_MARGIN = 0.1 * u.deg

log = logging.getLogger(__name__)

## Idea

The Evaluator is initiated with only the model. At the first call the `update` method is called with the events, IRFs, mask, pointing, etc...  
The IRFs and events do not need to be stored because only the model, the integration geom and the mask are necessay together with the calculated IRF cube and acceptance cube.

We copy from the `MapEvaluator` and adapt its functionalities.

In [21]:
class UnbinnedEvaluator:
    """Sky model evaluation at events' coordinates.
    Evaluates a sky model at the events's coordinates and returns a map of the predicted counts.
    Convolution with IRFs will be performed as defined in the sky_model
    Parameters
    ----------
    model : `~gammapy.modeling.models.SkyModel`
        Sky model
    exposure : `~gammapy.maps.Map` 
        Exposure map
    psf : `~gammapy.irf.PSFKernel`
        PSF kernel
    edisp : `~gammapy.irf.EDispKernel`
        Energy dispersion
    mask : `~gammapy.maps.Map`
        Mask to apply to the likelihood for fitting.
    gti : `~gammapy.data.GTI`
        GTI of the observation or union of GTI if it is a stacked observation
    evaluation_mode : {"local", "global"}
        Model evaluation mode.
        The "local" mode evaluates the model components on smaller grids to save computation time.
        This mode is recommended for local optimization algorithms.
        The "global" evaluation mode evaluates the model components on the full map.
        This mode is recommended for global optimization algorithms.
    use_cache : bool
        Use npred caching.
    """

    def __init__(
        self,
        model,
        events=None,
        mask = None,
        exposure= None,
        gti = None,
        evaluation_mode="local",
        use_cache=True,
        emin=None,
        emax=None,
        ebpd=8,
        spatialbs=0.02*u.deg
    ):

        self.model = model
        self.events = events
        self.mask = mask
        self.exposure= exposure
        self.gti = gti
        self.use_cache = use_cache
        self._init_position = None
        self.contributes = True
        self.psf_containment = None
        self.emin = emin
        self.emax = emax
        self.ebpd = ebpd
        self.spatialbs = spatialbs
        self.geom = None
        self.irf_cube = None
        
        if self.mask is None:
            log.warning(f"No mask given for model {self.model.name}.")
            if self.emin is None:              
                self.emin=0.1*u.TeV # some default values if no mask is defined which gives clues about the integration range
                log.warning("Setting the energy integration range emin to 0.1 TeV")
            if self.emax is None:
                self.emax=100*u.TeV
                log.warning("Setting the energy integration range emax to 100 TeV")
        
        if evaluation_mode not in {"local", "global"}:
            raise ValueError(f"Invalid evaluation_mode: {evaluation_mode!r}")

        self.evaluation_mode = evaluation_mode

        # TODO: this is preliminary solution until we have further unified the model handling
        if (
            isinstance(self.model, TemplateNPredModel)
            or self.model.spatial_model is None
            or self.model.evaluation_radius is None
        ):
            self.evaluation_mode = "global"

        # define cached computations
        self._cached_parameter_values = None
        self._cached_parameter_values_previous = None
        self._cached_parameter_values_spatial = None
        self._cached_position = (0, 0)
        self._computation_cache = None
        self._neval = 0  # for debugging
        self._renorm = 1
        self._spatial_oversampling_factor = 1
        if self.exposure is not None:
            if not self.geom.is_region or self.geom.region is not None:
                self.update_spatial_oversampling_factor(self.geom)

    def reset_cache_properties(self):
        """Reset cached properties."""
        del self._compute_npred
        del self._compute_flux_spatial

    # just use the exposure geometry because of edisp problems otherwise.
    def _init_geom(self, exposure=None, edisp=None):
        """True energy map geometry (`~gammapy.maps.Geom`) on which the model will be integrated"""
        
        if self.emin is None or self.emax is None:
            if exposure:
                e_edges = exposure.geom.axes['energy_true'].edges
                emin_exp, emax_exp = e_edges[0], e_edges[-1]
            else: emin_exp, emax_exp = None, None
            # check if exp bounds make sense regarding the mask
            idx=self.mask.geom.axes.index("energy")
            other_idx=np.arange(len(self.mask.geom.data_shape))
            other_idx=np.delete(other_idx, idx)
            edata = np.logical_or.reduce(self.mask.data, axis=tuple(other_idx))
            eedges = self.mask.geom.axes['energy'].edges
            if edisp:
                migra_min, migra_max=edisp.edisp_map.geom.axes['migra'].edges[[0,-1]]
            else: migra_min, migra_max = 1.0 , 1.0
        
            # choose the lower or upper bin of the mask (extended by the edisp)
            # if emin or emax is defined it is not changed here
            self.emin=self.emin or emin_exp or eedges[np.where(edata)[0].min()] * migra_min  
            self.emax=self.emax or emax_exp or eedges[np.where(edata)[0].max()+1] * migra_max 
        
        energy_axis_true = MapAxis.from_energy_bounds(
            self.emin,self.emax,self.ebpd, per_decade=True, name="energy_true"
        )
        if self.model.evaluation_radius is None or self.evaluation_mode == 'global':
            if self.exposure is not None:
                width=self.exposure.geom.width
            elif self.mask is not None:
                width=self.mask.geom.width
            else:
                width=1*u.deg
        else:
            width = self.model.evaluation_radius
            
        if self.spatialbs > self.model.evaluation_bin_size_min:
            log.warning(f"Setting the spatial bin size from {self.spatialbs:.3g} \
to the `model.evaluation_bin_size_min` of {self.model.evaluation_bin_size_min:.3g}")
            self.spatialbs = self.model.evaluation_bin_size_min
            
        geom = WcsGeom.create(
            skydir=self.model.spatial_model.position,
            binsz=self.spatialbs,
            width=width,
            frame=self.mask.geom.frame,  # same frame as the mask (not sure it is important)
            proj=self.mask.geom.projection, # also use same projection
            axes=[energy_axis_true],
        )
        self.geom = geom

    @property
    def needs_update(self):
        """Check whether the model component has drifted away from its support."""
        # TODO: simplify and clean up
        if isinstance(self.model, TemplateNPredModel):
            return False
        elif not self.contributes:
            return False
        elif self.exposure is None:
            return True
        elif self.geom.is_region:
            return False
        elif self.evaluation_mode == "global" or self.model.evaluation_radius is None:
            return False
        elif not self.parameters_spatial_changed(reset=False):
            return False
        else:
            return self.irf_position_changed

    @property
    def psf_width(self):
        """Width of the PSF"""
        if self.psf is not None:
            psf_width = np.max(self.psf.psf_kernel_map.geom.width)
        else:
            psf_width = 0 * u.deg
        return psf_width

    def use_psf_containment(self, geom):
        """Use psf containment for point sources and circular regions"""
        if not geom.is_region:
            return False

        is_point_model = isinstance(self.model.spatial_model, PointSpatialModel)
        is_circle_region = isinstance(geom.region, CircleSkyRegion)
        return is_point_model & is_circle_region

    @property
    def cutout_width(self):
        """Cutout width for the model component"""
        return self.psf_width + 2 * (self.model.evaluation_radius + CUTOUT_MARGIN)

    def update(self, events, exposure, psf=None, edisp=None, mask=None, use_modelpos=False):
        """Update the integration geometry, the kernel cube and the acceptance cube of the EventEvaluator, based on the current position of the model component.
        Parameters
        ----------
        exposure : `~gammapy.maps.Map`
            Exposure map.
        psf : `gammapy.irf.PSFMap`
            PSF map.
        edisp : `gammapy.irf.EDispMap`
            Edisp map.
        geom : `WcsGeom`
            Counts geom
        mask : `~gammapy.maps.Map`
            Mask to apply to the likelihood for fitting.
        use_modelpos : bool
            Wether or not to evaluate the IRFs at the model position or at each skycoord of the integration geom 
        """
        # TODO: simplify and clean up
        log.debug("Updating model evaluator")
        
        if self.evaluation_mode == "local":
            self.contributes = self.model.contributes(mask=mask, margin=self.psf_width)
        if self.contributes:
            # 1. get the contributing events which are close enough to the model
            del self.event_mask
            coords = events.map_coord(mask.geom)
            events = events.select_row_subset(self.event_mask)
            
            if isinstance(self.model, TemplateNPredModel):
                # the TemplateNpredModel only needs to be interpolated at 
                # the events' coordinates. No IRF cube necessary.
                self.events = events
                self.mask = mask
                return
            
            # init the proper integration geometry
            self._init_geom(exposure, edisp)
            self.exposure = exposure.interp_to_geom(self.geom)
            if use_modelpos == True:
                position = self.model.position
            else: position=None
            # get the edisp kernel factors for each event
            if edisp:
                edisp_factors = make_edisp_factors(edisp, self.geom, events, position=position)
            else:
                edisp_factors = 1.0

            # get the psf kernel factors for each event
            if psf and self.model.spatial_model:
                psf_factors = make_psf_factors(psf, self.geom, events, position=position)
            else:
                psf_factors = 1.0

            # maybe float32 is enough precision here?
            # maybe use sparse matrix
            self.irf_cube = psf_factors * edisp_factors[:,:,None,None]
            
            if mask:
                self.acceptance = make_accpetance(self.geom, mask, edisp, psf, self.model.position)
            else: self.acceptance = 1.0
            
        self.reset_cache_properties()
        self._computation_cache = None
        self._cached_parameter_previous = None
    
    @lazyproperty
    def event_mask(self):
        """create a mask for events too far away from the model"""
        # the spatial part: separation from the model center
        separation = self.events.radec.separation(self.model.position)
        # TODO: Define an individual width for each event dependent on its energy
        mask_spatial = separation < self.cutout_width/2 
        
        # possibility for an energy or temporal mask
        
        return mask_spatial

    

    @lazyproperty
    def _compute_npred(self):
        """Compute npred"""
        if isinstance(self.model, TemplateNPredModel):
            npred = self.model.evaluate()
            # interpolate on the events
            coords = self.events.map_coord(self.mask.geom)
            response = npred.interp_by_coord(coords)
            total = np.sum(npred.data[self.mask.data])
            
        else:
            if not self.parameter_norm_only_changed:
                npred=self.model.integrate_geom(self.geom, self.gti) 
                npred *= self.exposure.quantity
                response = npred * self.irf_cube
                total = npred * self.acceptance
                axis_idx = np.arange(len(response.shape))
                axis_idx=np.delete(axis_idx, 0) # dim 0 needs to be the event axis
                response = response.quantity.to_value('').sum(axis=tuple(axis_idx))
                total = total.quantity.to_value('').sum() 
                self._computation_cache = response, total
            else:
                response, total = self._computation_cache
                response *= self.renorm()
                total *= self.renorm()
                self._computation_cache = response, total
        return response, total

    @property
    def apply_psf_after_edisp(self):
        return (
            self.psf is not None and "energy" in self.psf.psf_kernel_map.geom.axes.names
        )

    def compute_npred(self):
        """Evaluate model predicted counts.
        Returns
        -------
        npred : `~gammapy.maps.Map`
            Predicted counts on the map (in reco energy bins)
        """
        if self.parameters_changed or not self.use_cache:
            del self._compute_npred

        return self._compute_npred

    @property
    def parameters_changed(self):
        """Parameters changed"""
        values = self.model.parameters.value

        # TODO: possibly allow for a tolerance here?
        changed = ~np.all(self._cached_parameter_values == values)

        if changed:
            self._cached_parameter_values = values

        return changed

    @property
    def parameter_norm_only_changed(self):
        """Only norm parameter changed"""
        norm_only_changed = False
        idx = self._norm_idx
        values = self.model.parameters.value
        if idx and self._computation_cache is not None:
            changed = self._cached_parameter_values_previous == values
            norm_only_changed = sum(changed) == 1 and changed[idx]

        if not norm_only_changed:
            self._cached_parameter_values_previous = values
        return norm_only_changed

    def parameters_spatial_changed(self, reset=True):
        """Parameters changed
        Parameters
        ----------
        reset : bool
            Reset cached values
        Returns
        -------
        changed : bool
            Whether spatial parameters changed.
        """
        values = self.model.spatial_model.parameters.value

        # TODO: possibly allow for a tolerance here?
        changed = ~np.all(self._cached_parameter_values_spatial == values)

        if changed and reset:
            self._cached_parameter_values_spatial = values

        return changed

    @property
    def irf_position_changed(self):
        """Position for IRF changed"""

        # Here we do not use SkyCoord.separation to improve performance
        # (it avoids equivalence comparisons for frame and units)
        lon_cached, lat_cached = self._cached_position
        lon, lat = self.model.position_lonlat

        separation = angular_separation(lon, lat, lon_cached, lat_cached)
        changed = separation > (self.model.evaluation_radius + CUTOUT_MARGIN).to_value(
            u.rad
        )

        if changed:
            self._cached_position = lon, lat

        return changed

    @lazyproperty
    def _norm_idx(self):
        """norm index"""
        names = self.model.parameters.names
        ind = [idx for idx, name in enumerate(names) if name in ["norm", "amplitude"]]
        if len(ind) == 1:
            return ind[0]
        else:
            return None

    def renorm(self):
        value = self.model.parameters.value[self._norm_idx]
        value_cached = self._cached_parameter_values_previous[self._norm_idx]
        return value / value_cached

    def peek(self, figsize=(12, 15)):
        """Quick-look summary plots.
        Parameters
        ----------
        figsize : tuple
            Size of the figure.
        """
        if self.needs_update:
            raise AttributeError(
                "The evaluator needs to be updated first. Execute "
                "`MapDataset.npred_signal(model_name=...)` before calling this method."
            )

        nrows = 1
        if self.psf:
            nrows += 1
        if self.edisp:
            nrows += 1

        fig, axes = plt.subplots(
            ncols=2,
            nrows=nrows,
            subplot_kw={"projection": self.exposure.geom.wcs},
            figsize=figsize,
            gridspec_kw={"hspace": 0.2, "wspace": 0.3},
        )

        axes = axes.flat

        exposure = self.exposure
        if isinstance(exposure, WcsNDMap) or isinstance(exposure, HpxNDMap):
            axes[0].set_title("Predicted counts")
            self.compute_npred().sum_over_axes().plot(ax=axes[0], add_cbar=True)

            axes[1].set_title("Exposure")
            self.exposure.sum_over_axes().plot(ax=axes[1], add_cbar=True)
        elif isinstance(exposure, RegionNDMap):
            axes[0].remove()
            ax0 = fig.add_subplot(nrows, 2, 1)
            ax0.set_title("Predicted counts")
            self.compute_npred().plot(ax=ax0)

            axes[1].remove()
            ax1 = fig.add_subplot(nrows, 2, 2)
            ax1.set_title("Exposure")
            self.exposure.plot(ax=ax1)

        idx = 3
        if self.psf:
            axes[2].set_title("Energy-integrated PSF kernel")
            self.psf.plot_kernel(ax=axes[2], add_cbar=True)

            axes[3].set_title("PSF kernel at 1 TeV")
            self.psf.plot_kernel(ax=axes[3], add_cbar=True, energy=1 * u.TeV)

            idx += 2

        if self.edisp:
            axes[idx - 1].remove()
            ax = fig.add_subplot(nrows, 2, idx)
            ax.set_title("Energy bias")
            self.edisp.plot_bias(ax=ax)

            axes[idx].remove()
            ax = fig.add_subplot(nrows, 2, idx + 1)
            ax.set_title("Energy dispersion matrix")
            self.edisp.plot_matrix(ax=ax)

In [13]:
from gammapy.data import DataStore, Observation
from gammapy.datasets import MapDataset, MapDatasetEventSampler, Datasets
from gammapy.makers import MapDatasetMaker, FoVBackgroundMaker, SafeMaskMaker
from astropy.coordinates import SkyCoord
from gammapy.modeling.models import (
#     Model,
#     Models,
    SkyModel,
#     PowerLawSpectralModel,
#     PowerLawNormSpectralModel,
#     PointSpatialModel,
    LogParabolaSpectralModel,
    GaussianSpatialModel,
#     TemplateSpatialModel,
#     ExpDecayTemporalModel,
#     LightCurveTemplateTemporalModel,
    FoVBackgroundModel,
)

In [14]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

In [15]:
obs_id = [23523] # just one observation 
obs1 = data_store.get_observations(obs_id)[0]
# selection = dict(
#     type="sky_circle",
#     frame="icrs",
#     lon="83.633 deg",
#     lat="22.014 deg",
#     radius="5 deg",
# )
# selected_obs_table = data_store.obs_table.select_observations(selection)

In [16]:
crab_pos = SkyCoord(184.557, -5.784, unit='deg', frame='galactic') 
obs_pos = obs1.pointing_radec
# choose energy binning
ebins = np.logspace(-1,2,49)
ebins_true = np.logspace(-1,2,49)

energy_axis = MapAxis.from_edges(
    ebins, unit="TeV", name="energy", interp="log"  
)
energy_axis_true = MapAxis.from_edges(
    ebins_true, unit="TeV", name="energy_true", interp="log"  
)
migra_axis = MapAxis.from_bounds(
    0.2, 5, nbin=160, node_type="edges", name="migra"
)
geom = WcsGeom.create(
    skydir=crab_pos,
    binsz=0.02,
    width=(3.5, 3.5),
    frame="icrs",  # same frame as events
    proj="CAR",
    axes=[energy_axis],
)

circle = CircleSkyRegion(
    center=crab_pos, radius=0.3 * u.deg
)
data = geom.region_mask(regions=[circle], inside=False)
exclusion_mask = ~geom.region_mask(regions=[circle])
maker_fov = FoVBackgroundMaker(method="fit", exclusion_mask=exclusion_mask)

In [17]:
%%time
maker = MapDatasetMaker(background_oversampling=2)
maker_safe_mask = SafeMaskMaker(methods=['offset-max'], offset_max='1.5 deg')
# providing the migra axis seems essential so that edisp is a EdispMap and no EdispKernelMap
reference = MapDataset.create(geom=geom, energy_axis_true=energy_axis_true, migra_axis=migra_axis)  

dataset = maker.run(reference, obs1)
dataset = maker_safe_mask.run(dataset, obs1)
dataset.mask_safe *= geom.energy_mask(energy_min=1*u.TeV)
# assert np.isfinite(dataset.background.data[dataset.mask_safe.data]).all()
# dataset.background.data[~dataset.mask_safe.data] = 0.0

bkg_model = FoVBackgroundModel(dataset_name=dataset.name)
dataset.models=bkg_model
# dataset.background_model.spectral_model.tilt.frozen = False
# dataset = maker_fov.run(dataset)
# print(
#     f"Background norm obs {obs1.obs_id}: {dataset.background_model.spectral_model.norm.value:.2f} \
#     (tilt={dataset.background_model.spectral_model.tilt.value:.2f})"
# )

CPU times: user 2.5 s, sys: 273 ms, total: 2.77 s
Wall time: 2.79 s


In [18]:
model_gauss = SkyModel(
    spatial_model=GaussianSpatialModel(lon_0="184.557 deg", lat_0="-5.784 deg", sigma='0.016 deg', frame = 'galactic'),
    spectral_model=LogParabolaSpectralModel(amplitude='3.5e-11 cm-2 s-1 TeV-1', 
                                          reference='1 TeV', 
                                          alpha=1.8, 
                                          beta=0.4
                                         ),
    name='crab_model_gauss'
    )

In [22]:
test=UnbinnedEvaluator(model_gauss, mask=dataset.mask_safe)
test.mask = dataset.mask_safe

In [23]:
test._init_geom(exposure=dataset.exposure, edisp=dataset.edisp)

Setting the spatial bin size from 0.02 deg to the `model.evaluation_bin_size_min` of 0.00533 deg


In [24]:
test.geom

WcsGeom

	axes       : ['lon', 'lat', 'energy_true']
	shape      : (15, 15, 24)
	ndim       : 3
	frame      : icrs
	projection : CAR
	center     : 83.6 deg, 22.0 deg
	width      : 0.1 deg x 0.1 deg
	wcs ref    : 83.6 deg, 22.0 deg

In [25]:
dataset.exposure.interp_to_geom(test.geom)

WcsNDMap

	geom  : WcsGeom 
 	axes  : ['lon', 'lat', 'energy_true']
	shape : (15, 15, 24)
	ndim  : 3
	unit  : m2 s
	dtype : float64

In [28]:
dataset.edisp.edisp_map.quantity

<Quantity [[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             ...,
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

            [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
              0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
             [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
   